# Задача 2.2
Рассмотрим фирму, которая занимается продажей лотерейных билетов. Правила лотереи следующее: все билеты являются выигрышными с вероятностью p. Билеты продаются до тех пор, пока хоть один человек не выиграет (гарантируется, что как только билет купили и он выигрышный, больше билетов не продают). Из-за вспышки коронавируса все заводы закрылись, а с ними и знание заветного p также пропало. Фирма хочет восстановить p, имея отчет о продажах билетов за последнии 5 месяцев: 8 билетов, 12 билетов, 7 билетов, 6 билетов и 12 билетов.

Требуется:

 - Оцените методом максимального правдоподобия параметр p0.
 
Теперь фирма нашла на складе несколько ящиков c билетами, которые вы можете использовать для проверки гипотезы о том, что истинное p равняется p0 — оценке максимального правдоподобия из предыдущего пункта. Для проверки был предложен следующий эксперимент: последовательно вскрываются n = 100 билетов, и проводился подсчет: сколько выигрышных билетов было из данных N штук. Данный эксперимент проводился 10 раз и были получены следующие результаты: 13, 8, 11, 10, 11, 12, 7, 9, 10, 9.

 - записать задачу формально;
 - предложить статистику для решения данной задачи;
 - получить приближенно нулевое распределение данной статистики;
 - записать явно правило принятия решения на основе статистики и нулевого распределения для обеспечения уровня значимости α = 0.05;
 - проверить гипотезу по записанному критерию, для данных из условия. Противоречат ли они гипотезу?

# Решение

## 1) Оценим параметр $p_0$ методом максимального правдоподобия

Пусть $X$ - число проданных лотерейных билетов. Тогда
$$P(X = x) = p(1-p)^{x-1},~x\in \mathbb{N}$$

Пусть было N экспериментов, в ходе которых продано $\bigl\{x_i\bigr\}_{i=1}^N$ билетов. Тогда для правдоподобия имеем:

$$\mathcal{L} = \prod_{i = 1}^{N} P(X = x_i) = \prod_{i = 1}^{N} p(1-p)^{x_i-1}$$

$$\log \mathcal{L} = N\log p + \sum_{i = 1}^{N}(x_i-1)\log (1-p) = N\log p + \Bigl(\sum_{i = 1}^{N}x_i - N\Bigr)\log (1-p)$$

$$\frac{\partial \log \mathcal{L}}{\partial p} = \frac{N}{p} - \frac{\sum_{i = 1}^{N}x_i - N}{1-p} = 0$$

$$p = \frac{1}{1+\frac{\sum_{i = 1}^{N}x_i - N}{N}} = \frac{N}{\sum_{i = 1}^{N}x_i} = \frac{1}{\bar x}$$

In [7]:
import numpy as np
p0 = lambda data: 1./data.mean()
data = np.array([8, 12, 7, 6, 12])
p0 = p0(data)
p0

0.1111111111111111

## 2) Сформулируем задачу и предложим статистику

В пусть в новом эксперименте $X$ - число выигрышных билетов. Для $X$ имеем биномиальное респределение с параметром $p$:

$$X \sim Bin(n, p),~\text{где р - вероятность билета оказаться выигрышным, а n=100}$$

Легко заметить, что если $X_i \sim Bin(n,p)$, то $\sum_{i=1}^N X_i \sim Bin(nN,p)$. 

Действительно, $X_i = \sum_{j=1}^n B_{ij}$, где $B_{ij} \sim Bernoulli(p)$, тогда $\sum_{i=1}^N X_i = \sum_{i=1}^N \sum_{j=1}^n B_{ij}\sim Bin(nN,p)$ как сумма $nN$ независимых бернуллиевских случайных величин.

Следовательно,
$$P\bigl( \sum_{i=1}^N x_i = T \bigr) = C_{nN}^T p^T(1-p)^{nN - T}$$

## 3) Получим приближенно нулевое распределение

Для получения приближенного распределения воспользуемся локальной теоремой Муавра-Лапласа:

$$P_n(m) \approx \frac{1}{\sqrt{2\pi npq}}\exp \Bigl(-\frac{x_m^2}{2}\Bigr),~~x_m = \frac{m-np}{\sqrt{npq}}$$

$$\text{Условия применимости:  } n>100,~m>20,~npq>10$$

В нашем случае 
$$n \to Nn = 1000 > 100$$

$$m \to \sum_{i=1}^{10} x_i = 100 > 20$$

$$npq \to p_0(1-p_0)Nn \approx 98.765 > 10$$


In [29]:
data = np.array([13, 8, 11, 10, 11, 12, 7, 9, 10, 9])
n = 100
N = len(data)
sum(data), p0*(1-p0)*n*N

(100, 98.76543209876543)

$$Bin(nN, T) = \frac{1}{\sqrt{2\pi Nnp_0(1-p_0)}}\exp \Bigl(-\frac{(T-nNp_0)^2}{2 Nnp_0(1-p_0)}\Bigr)$$

## 4) Таким образом, критерий формулируется так:

$$\text{выборка:   } X^N = (X_1\dots X_N),~~X \sim Bin(p, n)$$

$$\text{нулевая гипотеза:   } H_0:p=p_0$$

$$\text{альтернатива:   } H_1:p<\ne>p_0$$

$$\text{статистика:   } T(X^N) = \frac{\sum_{i=1}^N x_i-nNp_0}{\sqrt{Nnp_0(1-p_0)}}$$

$$\text{нулевое распределение:   } \sim \frac{\phi_{\mathscr N(0,1)}}{\sqrt{Nnp_0(1-p_0)}}$$

## 5) Получим правило для принятия решения (курильщика)

Если обозначить $S = \Bigl|\sum_{i=1}^N x_i - nNp_0 \Bigr|$, то для двусторонней альтернативы  достаточно большого $S$ имеем

$$p(T) = \sum_{|s| > S} P(s) \approx 2\sum_{s > S} P(s)$$

Можно показать, что $$\sum_{k=a}^{+\infty} f(k) - f(a) \le\int_a^{+\infty}f(x)dx \le \sum_{k=a}^{+\infty} f(k)$$

В нашем случае $$f(x) = \frac{\phi_{\mathscr N(0,1)}(x)}{\sqrt{Nnp_0(1-p_0)}}$$

$$\int_T^{+\infty}f(x)dx = 1-F_{\mathscr N(0,1)}(T) \text{ - при сжатии вдоль оси сделали замену переменных}$$

Таким образом, $$p(T) \in \Bigl[2(1-F_{\mathscr N(0,1)}(|T|));~~2(1-F_{\mathscr N(0,1)}(|T|)) + 2\frac{\phi_{\mathscr N(0,1)}(|T|)}{\sqrt{Nnp_0(1-p_0)}}\Bigr]$$


Отвергаем нулевую гипотезу в пользу альтернативы, если правая граница отрезка для p(T) < 0.05, т.е.

$$2(1-F_{\mathscr N(0,1)}(|T|)) + 2\frac{\phi_{\mathscr N(0,1)}(|T|)}{\sqrt{Nnp_0(1-p_0)}} < 0.05$$

In [13]:
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
def p(data = data, n = n, p0 = p0):
    T = (sum(data) - p0*n*len(data))/np.sqrt(p0*(1-p0)*n*len(data))
    s = 1./np.sqrt(p0*(1-p0)*n*len(data))
    return 2*(1-st.norm.cdf(np.abs(T))) + 2*st.norm.pdf(np.abs(T))*s

In [37]:
p()

0.30652625432099256

## 5)* Получим правило для принятия решения (здорового человека)

На самом деле, можно было воспользоваться интегральной теоремой Муавра - Лапласа.

$$\lim_{n \to \infty} P\Bigl(c \le \frac{X - np}{\sqrt{npq}} \le d\Bigr) = \frac{1}{\sqrt{2\pi}}\int_c^d e^{-\frac{x^2}{2}}dx,~~c = |T|, d = +\infty$$

$$p(T) = 2(1-F_{\mathscr N(0,1)}(|T|))\text{ - более мощный критерий}$$

In [39]:
T = (sum(data) - p0*n*len(data))/np.sqrt(p0*(1-p0)*n*len(data))
2*(1-st.norm.cdf(np.abs(T)))

0.2635524772829725

# Вывод
Данные не противоречат нулевой гипотезе. Видимо действительно $p = 0.11$